# Repeat Experiment 
## import packages

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from tcnn.utils.experiment.repeat import repeat_experiment

In [ ]:
from os.path import join
from config.config import load_config28

In [ ]:
dataset_name = "fracturemnist3d"
config = load_config28(dataset_name)

In [ ]:
ORDER = config["order"]
SIZE = config["data"]["size"]
TASK = config["task"]
NUM_CLASSES = config["network"]["num_classes"]
EPOCHS = config["train"]["epochs"]
BATCH_SIZE = config["data"]["batch_size"]
INPUT_CHANNEL = config["network"]["input_channels"]
LINEAR_SIZE = config["network"]["linear_size"]
CRITERION = config["train"]["criterion"]
DEVICE = config["train"]["device"]
NUM_EXPERIMENTS = config["repeat"]["num_experiments"]
EPOCHS_PER_EXPERIMENT = config["repeat"]["epochs_per_experiemnt"]
experiment_name = (
    f"repeat-{ORDER}-{dataset_name}-{SIZE}-{NUM_EXPERIMENTS}-{EPOCHS_PER_EXPERIMENT}"
)

In [ ]:
checkpoint_save_dir = join(config["train"]["checkpoint_save_dir"], experiment_name)
log_save_dir = join(config["repeat"]["log_save_dir"])

## get dataset

In [ ]:
from data_load import get_dataset, get_reset_dataloader

train_dataset, test_dataset = get_dataset(dataset_name, BATCH_SIZE, SIZE, TASK)

## reset dataloader

In [ ]:
reset_dataloader = get_reset_dataloader(TASK)

## import models

In [ ]:
from tcnn.utils.experiment.model import get_optimizers_medmnist, get_schedulers_medmnist
import lenet_models
import lenet_models_binary

if TASK == "multiclass":
    model_dict = lenet_models.get_model_dict(INPUT_CHANNEL, NUM_CLASSES, LINEAR_SIZE)
    print("Loaded multiclass model with {} classes".format(NUM_CLASSES))
elif TASK == "binary":
    model_dict = lenet_models_binary.get_model_dict(INPUT_CHANNEL, LINEAR_SIZE)
    print("Loaded binary model")

def_optimizer = get_optimizers_medmnist
def_scheduler = get_schedulers_medmnist

## define result dict

In [ ]:
import torch

results = dict()

## repeat experiment

In [ ]:
for model_name, model in model_dict.items():
    print("Traing model: ", model_name)
    results[model_name] = repeat_experiment(
        model,
        [train_dataset, test_dataset],
        reset_dataloader,
        BATCH_SIZE,
        CRITERION,
        def_optimizer,
        def_scheduler,
        NUM_EXPERIMENTS,
        EPOCHS_PER_EXPERIMENT,
        experiment_name + "-" + model_name,
        checkpoint_save=True,
        checkpoint_save_dir=checkpoint_save_dir,
        task=TASK,
    )
    with torch.no_grad():
        torch.cuda.empty_cache()
    print("***" * 10 + f"{model_name} done" + "***" * 10)

In [ ]:
from tcnn.utils.experiment.log import show_repeat_result, save_result

show_repeat_result(results)
save_result(results, save_name=experiment_name)

In [ ]:
from tcnn.utils.experiment.plot import plot_experiment_errorbar

plot_experiment_errorbar(
    results, metric_key="accuracy", baseline_key="lenet", ylabel="Accuracy"
)

In [ ]:
plot_experiment_errorbar(
    results, metric_key="accuracy", baseline_key="lenet_relu", ylabel="Accuracy"
)

In [ ]:
plot_experiment_errorbar(
    results, metric_key="auc_score", baseline_key="lenet", ylabel="AUC Score"
)

In [ ]:
plot_experiment_errorbar(
    results, metric_key="auc_score", baseline_key="lenet_relu", ylabel="AUC Score"
)

In [ ]:
plot_experiment_errorbar(
    results, metric_key="f1", baseline_key="lenet", ylabel="F1 Score"
)

In [ ]:
plot_experiment_errorbar(
    results, metric_key="f1", baseline_key="lenet_relu", ylabel="F1 Score"
)

In [ ]:
import torch

with torch.no_grad():
    torch.cuda.empty_cache()